# Charts Histograms

In [1]:
#  !pip install geemap 

In [2]:
import ee
import geemap
import geemap.chart as chart
import os
import folium

In [3]:
geemap.ee_initialize()

## Área de estudio

In [4]:
work_dir = os.path.expanduser('../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

## Funciones

In [5]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [6]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [7]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)

In [8]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B2','B4']).rename('NDWI')
    return image.addBands(ndwi)

In [9]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B2','B3']).rename('NDGI')
    return image.addBands(ndgi)

In [10]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [11]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [12]:
def llenado(image):
    fill = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.where(fill.select(['cloud']).gt(30),greenest98)

In [13]:
def muestra(image):
    muestras = image.sample(patagonia_grande, 10000)

## Colecciones

In [14]:
start_date = '1996-10-01'
end_date = '1997-03-31'

l5_1996 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [15]:
coleccion1996 = l5_1996.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [16]:
NDVI_1996 = coleccion1996.qualityMosaic('NDVI')

In [17]:
coleccion96 = coleccion1996.map(Cloud30)
greenest96 = coleccion96.qualityMosaic('NDVI')
map_1996 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1996.add_ee_layer(greenest96, vizParams, 'Mapa sin Nubes')
display(map_1996)

In [18]:
muestra = greenest96.sample(patagonia_grande, 10000)

In [19]:
property = 'date'

In [20]:
options = {"title": 'Fechas Greenest 96',
          "xlabel": 'Date',
          "ylabel": 'Pixel count',
          "colors": ['#1d6b99']
          }

In [21]:
chart.feature_histogram(muestra, property, maxBuckets=30 , **options)

## Llenado

In [22]:
start_date = '1998-10-01'
end_date = '1999-03-31'

l5_1998 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [23]:
coleccion1998 = l5_1998.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [24]:
NDVI_1998 = coleccion1998.qualityMosaic('NDVI')

In [25]:
coleccion98 = coleccion1998.map(Cloud30)
greenest98 = coleccion98.qualityMosaic('NDVI')
map_1998 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1998.add_ee_layer(greenest98, vizParams, 'Mapa 1998 sin Nubes')
display(map_1998)


In [26]:
muestra2 = greenest98.sample(patagonia_grande, 10000)

In [27]:
options2 = {"title": 'Fechas Greenest 98',
          "xlabel": 'Date',
          "ylabel": 'Pixel count',
          "colors": ['#EA1125']
          }

In [28]:
chart.feature_histogram(muestra2, property, maxBuckets=30, **options2)

In [29]:
mosaico = coleccion1996.map(llenado)

In [30]:
mosaico90= mosaico.qualityMosaic('NDVI')

In [31]:
Map = geemap.Map()

In [32]:
Map.addLayer(mosaico90, vizParams,'Mosiaco')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [33]:
muestra3 = mosaico90.sample(patagonia_grande, 5000)

In [34]:
options3 = {"title": 'Fechas Composición',
          "xlabel": 'Date',
          "ylabel": 'Pixel count',
          "colors": ['#1BD4C3']
          }

In [35]:
chart.feature_histogram(muestra3, property, maxBuckets=30, **options3)